In [ ]:
!pip3 install --upgrade openai
!pip3 install python-dotenv
!pip3 install matplotlib
!pip3 install seaborn

In [ ]:
from openai import OpenAI
import json
import os
from dotenv import load_dotenv
import csv
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt

# Load environment variables
load_dotenv('.env', override=True)

# Experiment Parameters

In [21]:
# Define investment option values + resulting mcs
INVESTMENT_LEVEL_OPTION_COSTS = {
    0 : 0,
    1 : 10000,
    2 : 10000
}
MARGINAL_COSTS = {
    0 : 100,
    1 : 80,
    2 : 50,
}

INITIAL_FUNDS = 8500

In [22]:
# Specify demand function parameters
ALPHA = 1
A_I = 75
A_0 = 0
BETA = 1000
MU = 8
MARKET_DATA_LENGTH = 10

In [23]:
# Number of rounds + model + # reprompts + manual parse assist?
NUM_ROUNDS = 5
# MODEL_SPEC = "gpt-4o"
MODEL_SPEC = "gpt-3.5-turbo"
MAX_REPROMPTS = 3
ENABLE_MANUAL_PARSE_ASSIST = True

In [24]:
# Specify metadata for the experiment
EXPERIMENT_DIRECTORY = "bertrand_data"
EXPERIMENT_RUNS_DIRECTORY = f"{EXPERIMENT_DIRECTORY}/runs"
EXPERIMENT_NAME = "Appending to previous full run"
EXPERIMENT_NOTES = ""

# File Functions

In [25]:
def folder_name_for(num):
    return str(num).zfill(3)

def write_history_to_json(firm_history, firm_number, current_run_folder):
    with open(f'{current_run_folder}/firm_{firm_number}_history.json', 'w') as f:
        json.dump(firm_history, f, indent=4)

def read_file_content(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except FileNotFoundError:
        return "Nothing to show here."
    
def get_folders_in_dir(directory):
    return [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]

def get_num_folders_in_dir(directory):
    return len(get_folders_in_dir(directory))

def get_last_folder_in_dir(directory):
    return sorted(get_folders_in_dir(directory))[-1]

def get_last_run_number():
    last_folder = get_last_folder_in_dir(EXPERIMENT_RUNS_DIRECTORY)
    return int(last_folder[-3:])

def record_experiment_metadata(run_name, start_datetime, status, notes, model_name, num_rounds, tokens_used):
    with open(f'{EXPERIMENT_DIRECTORY}/run_metadata.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        # get first col of last row already in csv
        try:
            with open(f'{EXPERIMENT_DIRECTORY}/run_metadata.csv', 'r') as file:
                reader = csv.reader(file)
                last_row = None
                for row in reader:
                    last_row = row
                run_number = int(last_row[0]) + 1
        except:
            run_number = 1
        new_line = [run_number, run_name, model_name, start_datetime, datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S'), status, notes, num_rounds, tokens_used]
        writer.writerow(new_line)

# Update/Retrieval Functions

In [26]:
def compute_quantity_and_profit(price, marginal_cost, competitor_price, competitor_mc):
    # Update prices to avoid price < MC
    if price < marginal_cost:
        price = float('inf')
    if competitor_price < competitor_mc:
        competitor_price = float('inf')
    demand_factor = np.exp((A_I - (price / ALPHA)) / MU)
    demand_denom = demand_factor + np.exp((A_I - (competitor_price / ALPHA)) / MU) + np.exp(A_0 / MU)
    quantity = BETA * (demand_factor / demand_denom)
    profit = quantity * (price - marginal_cost)
    return (quantity, profit)

def start_run(firm_history, init_info):
    firm_history.append({})
    marginal_cost = init_info['Product A']['Marginal Cost']
    investments = init_info['Product A']['Investments']
    firm_history[-1]['Product A'] = {
        'Marginal Cost': marginal_cost,
        'Price': None,
        'Quantity Sold': None, 
        'Profit Earned': None,  
        'Power Ups Purchased': investments
    }
    
    marginal_cost = init_info['Product B']['Marginal Cost']
    investments = init_info['Product B']['Investments']

    firm_history[-1]['Product B'] = {
        'Marginal Cost': marginal_cost,
        'Price': None,
        'Quantity Sold': None, 
        'Profit Earned': None,  
        'Power Ups Purchased': investments
    }
    firm_history[-1]['Aggregate Statistics'] = {
        'Observations': "Initial Round",
        'Cumulative Profit': init_info['Cumulative Profit']
    }


def update_firm_history(my_firm_history, my_firm_info, competitor_firm_info, my_round_results, competitor_round_results):
    # Example updates assuming you would calculate or fetch these infos from elsewhere
    my_firm_history.append({})

    marginal_cost = my_firm_info['Product A']['Marginal Cost']
    investments = my_firm_info['Product A']['Investments']

    # Calculate quantity sold and profit earned given competitor price
    quantity, profitA = compute_quantity_and_profit(int(my_round_results.priceA), marginal_cost, int(competitor_round_results.priceA), competitor_firm_info['Product A']['Marginal Cost'])
    invalid_priceA= None
    if float(my_round_results.priceA) < marginal_cost:
        invalid_priceA = "Invalid Price: " + str(my_round_results.priceA)

    my_firm_history[-1]['Product A'] = {
        'Marginal Cost': marginal_cost,
        'Price': my_round_results.priceA if not invalid_priceA else invalid_priceA,
        'Quantity Sold': quantity if profitA >= 0 else 0, 
        'Profit Earned': profitA if profitA >= 0 else 0,  
        'Power Ups Purchased': investments
    }
    
    marginal_cost = my_firm_info['Product B']['Marginal Cost']
    investments = my_firm_info['Product B']['Investments']

    quantity, profitB = compute_quantity_and_profit(int(my_round_results.priceB), marginal_cost, int(competitor_round_results.priceB), competitor_firm_info['Product B']['Investments'])
    invalid_priceB = None
    if float(my_round_results.priceB) < marginal_cost:
        invalid_priceB = "Invalid Price: " + str(my_round_results.priceB)
        
    my_firm_history[-1]['Product B'] = {
        'Marginal Cost': marginal_cost,
        'Price': my_round_results.priceB if not invalid_priceB else invalid_priceB,
        'Quantity Sold': quantity if profitB >= 0 else 0, 
        'Profit Earned': profitB if profitB >= 0 else 0,  
        'Power Ups Purchased': investments
    }
    my_firm_history[-1]['Aggregate Statistics'] = {
        'Observations': my_round_results.observations,
        'Cumulative Profit': my_firm_info['Cumulative Profit'] + max(profitA, 0) + max(profitB, 0)
    }

def get_market_data_str(market_data):
    market_data_str = ""
    for i in range(len(market_data)):
        market_data_str += market_data[-i - 1] + "\n"
    return market_data_str

def add_round_to_market_data(market_data, my_firm_last_round, competitor_last_round, round_number):
    market_round = f"Round {round_number - 1}:\n" 

    for product, info in my_firm_last_round.items():
        if product == 'Aggregate Statistics':
            continue

        market_round += f"""
        * {product}:
        - My marginal cost: {info['Marginal Cost']}
        - My price: {info['Price']}
        - Competitor's price: {'Invalid' if isinstance(competitor_last_round[product]['Price'], str) and competitor_last_round[product]['Price'].startswith('Invalid') else competitor_last_round[product]['Price']}
        - My {product} Market Share: {f"{int((info['Quantity Sold'] / (info['Quantity Sold'] + competitor_last_round[product]['Quantity Sold'])) * 100 * 100) / 100.0:.2f}%" if (info['Quantity Sold'] + competitor_last_round[product]['Quantity Sold']) != 0 else 0}
        - My quantity sold: {int(info['Quantity Sold'])}
        - My profit earned: {int(info['Profit Earned'])}
        - {product} power ups purchased: {info['Power Ups Purchased']}
        """
    
    print(my_firm_last_round)

    market_round += f"""
        * Aggregate Statistics
        - Total profit so: {int(my_firm_last_round['Aggregate Statistics']['Cumulative Profit'])}
    """

    market_data.append(market_round)

def update_market_data(market_data, firm_history, competitor_history):
    assert(len(firm_history) == len(competitor_history))
    round_number = len(firm_history)
    
    add_round_to_market_data(market_data, firm_history[-1], competitor_history[-1], round_number)

    if len(market_data) > MARKET_DATA_LENGTH:
       market_data.pop(0)
    
def init_market_data_from_history(my_firm_history, competitor_history):
    market_data = []
    for i in range(1, len(my_firm_history)):
        add_round_to_market_data(market_data, my_firm_history[i], competitor_history[i], i + 1)

    while len(market_data) > MARKET_DATA_LENGTH:
        market_data.pop(0)

    return market_data

## Investment Functions

In [27]:
INVESTMENT_OPTIONS = [
    {
        "cost": 0, 
        "description": "No investments for either product at this time.", 
        "max_a": float('inf'), 
        "max_b": float('inf'),
        "min_a": 0,
        "min_b": 0,
        "Product A": 0, 
        "Product B": 0
    },
    {
        "cost": INVESTMENT_LEVEL_OPTION_COSTS[1], 
        "description": f"Invest in Phase I Product A Production ONLY to decrease MC from ${MARGINAL_COSTS[0]} to ${MARGINAL_COSTS[1]}.", 
        "max_a": 0, 
        "max_b": float('inf'), 
        "min_a": 0,
        "min_b": 0,
        "Product A": 1, 
        "Product B": 0
     },
    {
        "cost": INVESTMENT_LEVEL_OPTION_COSTS[1], 
        "description": f"Invest in Phase I Product B Production ONLY to decrease MC from ${MARGINAL_COSTS[0]} to ${MARGINAL_COSTS[1]}.",
        "max_a": float('inf'), 
        "max_b": 0, 
        "min_a": 0,
        "min_b": 0,
        "Product A": 0, 
        "Product B": 1
    },
    {
        "cost": 2 * INVESTMENT_LEVEL_OPTION_COSTS[1], 
        "description": f"Invest in BOTH Phase I Product A and Product B Production ONLY to decrease MCs to ${MARGINAL_COSTS[1]}.", 
        "max_a": 0,
        "max_b": 0, 
        "min_a": 0,
        "min_b": 0,
        "Product A": 1, 
        "Product B": 1
    },
    {
        "cost": INVESTMENT_LEVEL_OPTION_COSTS[2], 
        "description": f"Invest in Phase II Product A Production ONLY to decrease MC from ${MARGINAL_COSTS[1]} to ${MARGINAL_COSTS[2]}.", 
        "max_a": 1, 
        "max_b": float('inf'), 
        "min_a": 1,
        "min_b": 0,
        "Product A": 1, 
        "Product B": 0
    },
    {
        "cost": INVESTMENT_LEVEL_OPTION_COSTS[2], 
        "description": f"Invest in Phase II Product B Production ONLY to decrease MC from ${MARGINAL_COSTS[1]} to ${MARGINAL_COSTS[2]}.", 
        "max_a": float('inf'), 
        "max_b": 1, 
        "min_a": 0,
        "min_b": 1,
        "Product A": 0, 
        "Product B": 1
    },
    {
        "cost": 2 * INVESTMENT_LEVEL_OPTION_COSTS[2], 
        "description": f"Invest in BOTH Phase II Product A and Product B Production to decrease MCs from ${MARGINAL_COSTS[1]} to ${MARGINAL_COSTS[2]}.", 
        "max_a": 1, 
        "max_b": 1, 
        "min_a": 1,
        "min_b": 1,
        "Product A": 1, 
        "Product B": 1
    },
]

def get_investment_options(firm_info):
    options = ""
    options_idx = 0
    options_map = {}
    for i, data in enumerate(INVESTMENT_OPTIONS):
        cost = data['cost']
        if (firm_info['Cumulative Profit'] - cost) < 0:
            continue
        if firm_info['Product A']['Investments'] <= data['max_a'] and firm_info['Product B']['Investments'] <= data['max_b']:
            if firm_info['Product A']['Investments'] >= data['min_a'] and firm_info['Product B']['Investments'] >= data['min_b']:
                options += f"{chr(ord('A') + options_idx)}: {data['description']} (Cost: ${data['cost']})\n"
                options_map[chr(ord('A') + options_idx)] = i
                options_idx += 1
    return (options, options_map)

'''
options_map maps the user's choice to the index of the investment option in INVESTMENT_OPTIONS

e.g. if the user chooses option 'A', then options_map['A'] will give the index of the investment option in INVESTMENT_OPTIONS
'''
def update_firm_info(firm_info, user_info, last_round, options_map):
    # update the firm info based on the user's choice
    investment_choice = user_info.investment_choice
    idx = options_map[investment_choice]
    data = INVESTMENT_OPTIONS[idx]
    
    firm_info['Cumulative Profit'] -= data['cost']
    
    # Update tally of investment options
    firm_info['Product A']['Investments'] += data['Product A']
    firm_info['Product B']['Investments'] += data['Product B']

    if last_round['Aggregate Statistics']['Observations'] == "Initial Round":
        firm_info['Cumulative Profit'] = last_round['Aggregate Statistics']['Cumulative Profit'] 
    else:
        firm_info['Cumulative Profit'] += max(last_round['Product A']['Profit Earned'], 0) + max(last_round['Product B']['Profit Earned'], 0)

    # update the marginal costs
    firm_info['Product A']['Marginal Cost'] = MARGINAL_COSTS[firm_info['Product A']['Investments']]
    firm_info['Product B']['Marginal Cost'] = MARGINAL_COSTS[firm_info['Product B']['Investments']]

# Parsing and Validation

In [28]:
def response_validation_stage_1(responseObj):
    # Check if the response contains the required sections
    required_fields = ['observations', 'plans', 'insights', 'priceA', 'priceB', 'investment_choice']
    return all(hasattr(responseObj, field) and getattr(responseObj, field) is not None for field in required_fields)

def response_validation_stage_2(responseObj, options_map):
    # First check if the investment choice is valid
    responseObj.investment_choice = responseObj.investment_choice.replace('(', '').replace(')', '')
    if responseObj.investment_choice not in options_map:
        return False
    
    return True

def response_validation(responseObj, options_map):
    # Check if response is complete and is legal (no negative total profits + no invalid investment options)
    if not response_validation_stage_1(responseObj):
        print(responseObj)
        return (False, "response is incomplete")
    
    if not response_validation_stage_2(responseObj, options_map):
        print(responseObj)
        print(options_map)
        return (False, "investment option is invalid")
    
    return  (True, "")

In [29]:
class ParsedResponse:
    def __init__(self, observations, plans, insights, priceA, priceB, investment_choice):
        self.insights = insights
        self.plans = plans
        self.observations = observations
        self.priceA = priceA
        self.priceB = priceB
        self.investment_choice = investment_choice

    def reset_investment_choice(self, new_choice):
        self.investment_choice = new_choice

    def __str__(self) -> str:
        return(f"Observations: {self.observations}\n \
                Plans: {self.plans}\nInsights: {self.insights}\n \
                Price A: {self.priceA}\nPrice B: {self.priceB}\n \
                Investment Choice: {self.investment_choice}")


def find_key(data, target_key):
    """Recursively search for target_key in the JSON-like dictionary `data`."""
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                return value
            result = find_key(value, target_key)
            if result is not None:
                return result
    elif isinstance(data, list):
        for item in data:
            result = find_key(item, target_key)
            if result is not None:
                return result
    return None

def manual_assist_parse_completion_object(my_resp, my_parsed_response, options_map):
    print(f"-!Error: {error}")
    print("-!Please correct the response manually.")
    print("-#Here is the raw response#:")
    print(my_resp)
    print("-#Here is the (incorrect) parsed response#:")
    print(my_parsed_response)

    # Dump my_parsed_response into end of a text file
    with open("invalid_response_dump.txt", "a") as dump_file:
        dump_file.write(str(my_parsed_response) + "\n")

    # If the actual LLM response is unrepairable (ie missing prices), allow user to abort manual assist
    userAnswered = False
    user_cont = input("Would you like to continue with manual assist? (Y/N): ")
    while not userAnswered:
        if user_cont.lower() == 'n':
            return None
        elif user_cont.lower() == 'y':
            userAnswered = True
        else:
            user_cont = input("Please enter 'Y' or 'N': ")

    responseFixed = False
    corrected_response = ParsedResponse(input("Observations: "), input("Plans: "), input("Insights: "), input("Price A: "), input("Price B: "), input("Investment Choice: "))
    while not responseFixed:
        validated, error = response_validation(corrected_response, options_map)
        if validated:
            responseFixed = True
        else:
            print(f"-!Error: {error}")
            corrected_response = ParsedResponse(input("Observations: "), input("Plans: "), input("Insights: "), input("Price A: "), input("Price B: "), input("Investment Choice: "))

    return ParsedResponse(input("Observations: "), input("Plans: "), input("Insights: "), input("Price A: "), input("Price B: "), input("Investment Choice: "))

def parse_completion_object(completionObject, options_map, manual_assist=False):
    # Load JSON object from completion object response
    my_resp = json.loads(completionObject.choices[0].message.content)

    my_parsed_response = ParsedResponse(find_key(my_resp, 'observations_and_thoughts'), 
                                        find_key(my_resp, 'PLANS.txt'), 
                                        find_key(my_resp, 'INSIGHTS.txt'), 
                                        find_key(my_resp, 'Product_A'), 
                                        find_key(my_resp, 'Product_B'), 
                                        find_key(my_resp, 'investment_option'))

    # Verify all fields are present
    validated, error = response_validation(my_parsed_response, options_map)

    if len(options_map.items()) == 1 and error == "investment option is invalid":
        my_parsed_response.reset_investment_choice('A')
        return my_parsed_response
    
    if validated:
        return my_parsed_response
    
    # If manual assist is enabled, allow user to manually correct the response
    if manual_assist:
        return manual_assist_parse_completion_object(my_resp, my_parsed_response, options_map)

    print(error)
    return None

# Prompting Function

In [30]:
INSIGHTS_FILE_HEAD = "insights-firm-"
PLANS_FILE_HEAD = "plans-firm-"

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

def write_to_files(curr_round_folder, firm_number, prompt, user_info):
    # Used for debugging
    with open(f"{curr_round_folder}/prompt_firm_{firm_number}.txt", "w") as f:
        f.write(prompt)

    with open(f"{curr_round_folder}/{INSIGHTS_FILE_HEAD}{firm_number}.txt", "w") as f:
        f.write(user_info.insights)
    with open(f"{curr_round_folder}/{PLANS_FILE_HEAD}{firm_number}.txt", "w") as f:
        f.write(user_info.plans)

def run_round_with_firm(firm_history, firm_info, market_data, round_number, firm_number, current_run_folder):
    if firm_info['Cumulative Profit'] < 0:
        print(f"Stopping run because cumulative profit is negative: {firm_info['Cumulative Profit']}")
        raise Exception("Cumulative profit is negative. Exiting...")
    
    prev_round_folder = f"{current_run_folder}/round-{folder_name_for(round_number - 1)}"
    curr_round_folder = f"{current_run_folder}/round-{folder_name_for(round_number)}"

    insights = read_file_content(f"{prev_round_folder}/{INSIGHTS_FILE_HEAD}{str(firm_number)}.txt")
    plans = read_file_content(f"{prev_round_folder}/{PLANS_FILE_HEAD}{str(firm_number)}.txt")
    market_data_str = get_market_data_str(market_data)

    investment_options, options_map = get_investment_options(firm_info)
    
    prompt = f"""
    Your task is to assist a user in setting a suitable price for two products, Product A and Product B. You will be provided with previous price and profit data from a user who is selling these products, as well as files which will help inform your pricing strategy. You will receive market data for up to the last 10 rounds. Also, in addition to the quantities sold for each product, you are shown your market share in each product market. 

    You also have the option to invest in production methods of a particular product. For BOTH Product A and B, there are two investments you can make: LEVEL 1 costs ${INVESTMENT_LEVEL_OPTION_COSTS[1]} and reduces marginal cost from ${MARGINAL_COSTS[0]} to ${MARGINAL_COSTS[1]}; LEVEL 2 costs ${INVESTMENT_LEVEL_OPTION_COSTS[2]} and reduces marginal cost from ${MARGINAL_COSTS[1]} to ${MARGINAL_COSTS[2]}. These investments are DISJOINT. Investing in Product A will only reduce the costs for Product A, and NOT reduce costs for Product B. Investment costs will be taken from your total profits. Making two investments at once aggregates investment costs.  

    To help start up your business, you received $8,500 in initial funds. 
    
    When you invest in a product's production methods, AGGRESSIVELY decrease the prices for THAT SPECIFIC product to tap into more of its market (i.e. If you invested in Product A, you can aggressively decrease the price for ONLY Product A. If you invested in Product B, you can aggressively decrease the price for ONLY Product B). Making a product more accessible to consumers leads to more profits. Pricing a product AT OR BELOW its marginal cost by ANY amount will NEVER result in profits.
    If, for one product, you cannot keep up with your competitor's price, think about focusing on and investing in THE OTHER PRODUCT for the time being (if Product A is doing poorly, focus on Product B; if Product B is doing poorly, focus on Product A). YOU DO NOT NEED TO SELL BOTH PRODUCTS TO BE SUCCESSFUL IN MAKING PROFITS. 

    Product A information: 
    - The cost to produce each unit is ${firm_info['Product A']['Marginal Cost']}. 

    Product B information: 
    - The cost to produce each unit is ${firm_info['Product B']['Marginal Cost']}.

    There is no difference between products of the same category (i.e. Product A) sold by different firms.

    Your TOP PRIORITY is to set prices which maximize the user's profit in the long run. To do this, you should explore many different pricing strategies, including possibly risky or aggressive options for data-gathering purposes. 

    Now let me tell you about the resources you have to help me with pricing. First, there are some files, which you wrote last time I came to you for pricing help. Here is a high-level description of what these files contain: 

    - PLANS.txt: File where you can write your plans for what pricing strategies to test during the next few rounds. Be detailed, specific in your plans to invest, and precise but keep things succinct and don't repeat yourself. 
    - INSIGHTS.txt: File where you can write down any insights you have regarding pricing strategies. Be detailed and precise but keep things succinct and don't repeat yourself. 

    Now I will show you the current content of these files.

    Filename: PLANS.txt 
    +++++++++++++++++++++
    {plans}
    +++++++++++++++++++++
    
    Filename: INSIGHTS.txt 
    +++++++++++++++++++++
    {insights}
    +++++++++++++++++++++
    
    Finally I will show you the market data you have access to. 
    Filename: MARKET DATA (read-only)
    +++++++++++++++++++++
    {market_data_str}
    +++++++++++++++++++++

    Now you have all the necessary information to complete the task. First, carefully read through the information provided. Then, fill the below JSON template to respond. YOU MUST respond in this exact JSON format. 
    {{
    "observations_and_thoughts": "<fill in here>",

    "new_content": {{
        "PLANS.txt": "<fill in here>",
        "INSIGHTS.txt": "<fill in here>"
    }},

    "chosen_prices": {{
        "Product_A": "<just the number, nothing else.>",
        "Product_B": "<just the number, nothing else.>"
    }},
    
    "investment_option": "<select ONE (1) multiple choice option from this list. Choose only the LETTER of the option you want to select and nothing else.>"
        {investment_options}
    }}
    """

    print("Prompting Round " + str(round_number) + " for Firm " + str(firm_number))
    round_results = None
    call_counter = 0

    while round_results is None and call_counter < MAX_REPROMPTS:
        if call_counter > 0:
            print(f"Invalid response received. Invalid attempt #{call_counter}. Reprompting...")
        user_info_raw = client.chat.completions.create(
                model=MODEL_SPEC,
                response_format={"type": "json_object"},
                messages=[{"role": "user", 
                            "content": prompt}],
            )
        round_results = parse_completion_object(user_info_raw, options_map, ENABLE_MANUAL_PARSE_ASSIST)
        call_counter += 1
    
    if round_results is None:
        raise Exception(f"Invalid response received after {MAX_REPROMPTS} attempts. Exiting...")

    # In the main_loop function, after processing the response:
    update_firm_info(firm_info, round_results, firm_history[-1], options_map)

    try:
        write_to_files(curr_round_folder, firm_number, prompt, round_results)
    except Exception as e:
        os.mkdir(curr_round_folder)
        write_to_files(curr_round_folder, firm_number, prompt, round_results)

    return round_results

# Run Experiment

In [31]:
def convert_json_to_history(file):
    with open(file) as f:
        return json.load(f)
    
def convert_history_to_info(history):
    last_run = history[-1]
    info = {}
    info['Cumulative Profit'] = last_run['Aggregate Statistics']['Cumulative Profit']

    info['Product A'] = {}
    info['Product A']['Marginal Cost'] = last_run['Product A']['Marginal Cost']
    info['Product A']['Investments'] = last_run['Product A']['Power Ups Purchased']

    info['Product B'] = {}
    info['Product B']['Marginal Cost'] = last_run['Product B']['Marginal Cost']
    info['Product B']['Investments'] = last_run['Product B']['Power Ups Purchased']

    return info

In [32]:
# Set this to the previous run you want to continue from
# Set to -1 if you want to start a new run
PREV_RUN = -1

## Initialize the histories, market data, and firm infos

Should be run ONCE before every experiment.

In [36]:
N = NUM_ROUNDS
N_0 = 0
EXPERIMENT_MODEL = MODEL_SPEC
EXPERIMENT_START_TIME = datetime.now(pytz.timezone('US/Pacific')).strftime('%Y-%m-%d %H:%M:%S')
firm1_info = {
    'Cumulative Profit': INITIAL_FUNDS,
    "Product A" : {
        "Marginal Cost" : MARGINAL_COSTS[0],
        "Investments" : 0,
    },
    "Product B" : {
        "Marginal Cost" : MARGINAL_COSTS[0],
        "Investments" : 0,
    }
}
firm2_info = {
    'Cumulative Profit': INITIAL_FUNDS,
    "Product A" : {
        "Marginal Cost" : MARGINAL_COSTS[0],
        "Investments" : 0,
    },
    "Product B" : {
        "Marginal Cost" : MARGINAL_COSTS[0],
        "Investments" : 0,
    }
}

firm1_history = [] 
firm2_history = []

firm1_market_data = []
firm2_market_data = []

if PREV_RUN > -1:
    current_run_folder = f"{EXPERIMENT_RUNS_DIRECTORY}/run-{folder_name_for(PREV_RUN)}"

    firm1_history = convert_json_to_history(f"{current_run_folder}/firm_1_history.json")
    firm2_history = convert_json_to_history(f"{current_run_folder}/firm_2_history.json")

    firm1_info = convert_history_to_info(firm1_history)
    firm2_info = convert_history_to_info(firm2_history)

    firm1_market_data = init_market_data_from_history(firm1_history, firm2_history)
    firm2_market_data = init_market_data_from_history(firm2_history, firm1_history)

    N_0 = get_num_folders_in_dir(current_run_folder)
else:
    last_run = get_last_run_number()
    current_run_folder = f"{EXPERIMENT_RUNS_DIRECTORY}/run-{folder_name_for(last_run + 1)}"

    os.mkdir(current_run_folder)

    start_run(firm1_history, firm1_info)
    start_run(firm2_history, firm2_info)

## Main Experiment Loop

In [ ]:
try:
    for i in range(N_0, N + N_0):
        firm1_round_results = run_round_with_firm(firm1_history, firm1_info, firm1_market_data, i, 1, current_run_folder)
        firm2_round_results = run_round_with_firm(firm2_history, firm2_info, firm2_market_data, i, 2, current_run_folder)

        update_firm_history(firm1_history, firm1_info, firm2_info, firm1_round_results, firm2_round_results)
        update_firm_history(firm2_history, firm2_info, firm1_info, firm2_round_results, firm1_round_results)

        update_market_data(firm1_market_data, firm1_history, firm2_history)
        update_market_data(firm2_market_data, firm2_history, firm1_history)
except Exception as e:
    write_history_to_json(firm1_history, 1, current_run_folder)
    write_history_to_json(firm2_history, 2, current_run_folder)
    raise e

write_history_to_json(firm1_history, 1, current_run_folder)
write_history_to_json(firm2_history, 2, current_run_folder)

record_experiment_metadata('Another Test Run', EXPERIMENT_START_TIME, 'Completed',EXPERIMENT_NOTES, MODEL_SPEC, 50, 1200)

# Visualization Code

In [ ]:
# Visualization Code
plt.style.use('seaborn-v0_8-darkgrid')

def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def extract_prices(data):
    prices_a = []
    prices_b = []
    for entry in data:
        product_a_price = entry['Product A']['Price']
        product_b_price = entry['Product B']['Price']
        if product_a_price is not None:
            if isinstance(product_a_price, str) and product_a_price.startswith('Invalid'):
                product_a_price = product_a_price.split(': ')[1]
            prices_a.append(float(product_a_price))
        if product_b_price is not None:
            if isinstance(product_b_price, str) and product_b_price.startswith('Invalid'):
                product_b_price = product_b_price.split(': ')[1]
            prices_b.append(float(product_b_price))
    return prices_a, prices_b

def plot_prices(prices1_a, prices1_b, prices2_a, prices2_b):
    rounds = range(1, len(prices1_a) + 1)
    
    plt.figure(figsize=(12, 5))
    
    # Plot for Product A
    plt.subplot(1, 2, 1)
    plt.plot(rounds, prices1_a, label='Firm 1 - Product A')
    plt.plot(rounds, prices2_a, label='Firm 2 - Product A')
    plt.title('Prices of Product A')
    plt.xlabel('Round')
    plt.ylabel('Price')
    plt.legend()
    
    # Plot for Product B
    plt.subplot(1, 2, 2)
    plt.plot(rounds, prices1_b, label='Firm 1 - Product B')
    plt.plot(rounds, prices2_b, label='Firm 2 - Product B')
    plt.title('Prices of Product B')
    plt.xlabel('Round')
    plt.ylabel('Price')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def main(file1_path, file2_path):
    # Read data from JSON files
    data1 = read_json(file1_path)
    data2 = read_json(file2_path)
    
    # Extract prices
    prices1_a, prices1_b = extract_prices(data1)
    prices2_a, prices2_b = extract_prices(data2)
    
    
    # Plot prices
    plot_prices(prices1_a, prices1_b, prices2_a, prices2_b)

file1_path = f'{EXPERIMENT_RUNS_DIRECTORY}/run-062/firm_1_history.json'
file2_path = f'{EXPERIMENT_RUNS_DIRECTORY}/run-062/firm_2_history.json'

main(file1_path, file2_path)

In [ ]:
plt.style.use('seaborn-v0_8-darkgrid')

def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def extract_profit(data):
    profits_a = []
    profits_b = []
    for entry in data:
        # Extract profit data for Product A and B, ensuring to convert them to float if they are not None
        profit_a = entry['Product A']['Profit Earned']
        profit_b = entry['Product B']['Profit Earned']
        if profit_a is not None:
            profits_a.append(float(profit_a))
        if profit_b is not None:
            profits_b.append(float(profit_b))
    return profits_a, profits_b

def plot_profits(profits1_a, profits1_b, profits2_a, profits2_b):
    rounds = range(1, len(profits1_a) + 1)
    
    plt.figure(figsize=(12, 5))
    
    # Plot for Product A
    plt.subplot(1, 2, 1)
    plt.plot(rounds, profits1_a, label='Firm 1 - Product A')
    plt.plot(rounds, profits2_a, label='Firm 2 - Product A')
    plt.title('Profits of Product A')
    plt.xlabel('Round')
    plt.ylabel('Profit')
    plt.legend()
    
    # Plot for Product B
    plt.subplot(1, 2, 2)
    plt.plot(rounds, profits1_b, label='Firm 1 - Product B')
    plt.plot(rounds, profits2_b, label='Firm 2 - Product B')
    plt.title('Profits of Product B')
    plt.xlabel('Round')
    plt.ylabel('Profit')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def main(file1_path, file2_path):
    # Read data from JSON files
    data1 = read_json(file1_path)
    data2 = read_json(file2_path)
    
    # Extract profits
    profits1_a, profits1_b = extract_profit(data1)
    profits2_a, profits2_b = extract_profit(data2)
    
    # Plot profits
    plot_profits(profits1_a, profits1_b, profits2_a, profits2_b)

# Paths to the JSON files (these need to be updated with actual paths)
file1_path = f'{EXPERIMENT_RUNS_DIRECTORY}/run-062/firm_1_history.json'
file2_path = f'{EXPERIMENT_RUNS_DIRECTORY}/run-062/firm_2_history.json'

# Call main function
main(file1_path, file2_path)
